In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
#from matplotlib_venn import venn3
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F
import torch
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import mat4py
import pandas as pd
import data_tracks
import models
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import KFold
from colorama import Fore
import pickle as pkl
import viz
from style import *
import analyze_helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# compare all models

In [6]:
out_dir = 'results/outcome=y_thresh'
results = analyze_helper.load_results(out_dir)

/accounts/grad/xsli/.local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/accounts/grad/xsli/.local/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


**look at prediction metrics**

In [7]:
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r = r.sort_values('accuracy', ascending=False)
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])

,accuracy,balanced_accuracy,f1,precision,recall,roc_auc
model_type,,,,,,
mlp2_11_none,0.676,0.626,0.768,0.707,0.859,0.71
svm_23_none,0.676,0.619,0.772,0.702,0.877,0.619
mlp2_9_none,0.675,0.631,0.765,0.712,0.845,0.715
svm_35_none,0.674,0.613,0.771,0.698,0.88,0.613
svm_39_none,0.673,0.615,0.771,0.698,0.881,0.615
svm_9_none,0.67,0.613,0.769,0.698,0.877,0.613
svm_11_none,0.668,0.61,0.767,0.697,0.873,0.61
logistic_39_none,0.668,0.624,0.759,0.708,0.839,0.702
logistic_4_none,0.667,0.619,0.766,0.703,0.873,0.7


**look at feat importances**

In [8]:
r = results
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r = r.sort_values('accuracy', ascending=False)
keys = [k for k in r if '_f' in k]
keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
# r = r[r.index.str.contains('39')]
# r = r[r.index.str.contains('11')]
r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})

r = r[r.index.str.contains('11')]
# r = r[r2.index.str.contains('ros')]
r = r[r.index.str.contains('none')]
r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r
# 
r = rank(r)
r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank
idxs = r.index
r.insert(0, 'acc', results.loc[idxs]['accuracy'])

subset = list(r.keys())
subset.remove('acc')
r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1, subset=subset) # rows differently colored

,acc,lifetime,fall,X_max,rise,X_min,X_mean,msd,td,max_diff,min_diff,down_max,left_max,nmf_0,nmf_10,nmf_11,nmf_1,nmf_2,nmf_3,nmf_4,nmf_5,nmf_6,nmf_7,nmf_8,nmf_9,right_max,sc_0,sc_10,sc_11,sc_1,sc_2,sc_3,sc_4,sc_5,sc_6,sc_7,sc_8,sc_9,up_max
model_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
mlp2_11_none,0.676,2,1,7,9,8,3,6,3,3,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
svm_11_none,0.668,2,1,3,5,4,6,8,6,10,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
logistic_11_none,0.662,1,7,2,4,2,5,6,8,9,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
rf_11_none,0.66,2,1,5,3,4,10,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
irf_11_none,0.639,1,3,4,2,6,5,7,8,10,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
gb_11_none,0.633,5,1,3,7,4,9,1,5,10,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dt_11_none,0.576,1,2,4,3,6,5,10,8,8,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# analyze model predictions

In [ ]:
df = data_tracks.get_data()
n = df.shape[0]

# make logistic data
# feat_names = ['X_max', 'lifetime', 'total_displacement', 'mean_square_displacement']
# feat_names = ['X_max', 'lifetime', 'total_displacement', 'mean_square_displacement', 'rise', 'fall']
# feat_names = ['X_max', 'lifetime', 'total_displacement', 'mean_square_displacement', 
#               'rise', 'fall', 'X_std', 'X_min', 'X_mean', 'max_diff', 'min_diff']
scs = [f'sc_{i}' for i in range(12)]
bases = [f'sc_{i}' for i in range(12)] + [f'nmf_{i}' for i in range(12)]
patches = ['up_max', 'down_max', 'left_max', 'right_max']
feat_names = ['X_max', 'lifetime', 'total_displacement', 'mean_square_displacement', 
              'rise', 'fall', 'X_std', 'X_min', 'X_mean', 'max_diff', 'min_diff'] + bases + patches
X = df[feat_names]

# normalize and store
X_mean = X.mean()
X_std = X.std()
norms = {
    feat_names[i]: {'mu': X_mean[i], 'std': X_std[i]} for i in range(len(feat_names))
}
X = (X - X_mean) / X_std
y = df['y_thresh'].values


# split testing data based on cell num
idxs_test = df.cell_num.isin([6])
X_test, Y_test = X[idxs_test], y[idxs_test]
X_traces_test = df.X[idxs_test]

### look at single model

In [68]:
# results = pkl.load(open('results/classify_outcome=y_consec_sig/rf_ros_11.pkl', 'rb'))
results_individual = pkl.load(open('results/outcome=y_thresh/mlp2_11_none.pkl', 'rb'))
preds, preds_proba = analyze_helper.analyze_individual_results(results_individual, X_test, Y_test, 
                                                print_results=False, plot_results=False)

In [ ]:
viz.viz_errs_lifetime(X_test, preds, preds_proba, Y_test, norms)

In [ ]:
viz.viz_errs_spatially(df, idxs_test, preds, Y_test)

In [ ]:
idxs_fp = preds > Y_test
idxs_fn = preds < Y_test
viz.viz_biggest_errs(X_traces_test[idxs_fp], Y_test[idxs_fp], preds[idxs_fp], preds_proba[idxs_fp])    
viz.viz_biggest_errs(X_traces_test[idxs_fn], Y_test[idxs_fn], preds[idxs_fn], preds_proba[idxs_fn])    

### look at many models

In [39]:
r = results
r = r.sort_values('accuracy', ascending=False)
idx = np.array(r.index)
print(idx[:10])
# model_names = ['mlp2_11_none', 'svm_35_none', 'logistic_4_none', 'rf_9_none']
model_names = ['mlp2_11_none', 'mlp2_9_none', 'mlp2_4_none']
model_names = idx[:10] #['mlp2_11_none', 'mlp2_9_none', 'mlp2_4_none']

d = {}
for i, model_name in enumerate(model_names):
    results_individual = pkl.load(open(f'results/outcome=y_thresh/{model_name}.pkl', 'rb'))
    preds, preds_proba = analyze_helper.analyze_individual_results(results_individual, X_test, Y_test, 
                                                print_results=False, plot_results=False)
    d[model_name] = preds
    d[model_name + '_proba'] = preds_proba
    d[model_name + '_errs'] = preds!=Y_test
df_preds = pd.DataFrame.from_dict(d)

['mlp2_11_none' 'mlp2_9_none' 'svm_35_none' 'svm_23_none' 'svm_9_none'
 'svm_11_none' 'logistic_4_none' 'svm_4_none' 'mlp2_4_none' 'rf_9_none']


In [40]:
# df_p = df_preds[[k for k in df_preds if 'proba' in k]]
# sns.pairplot(df_p) #, height=8)
# plt.show()

**venn-diagram**

In [ ]:
sets = []
for model_name in model_names[:3]:
    args = np.argwhere(df_preds[model_name + '_errs'])
    sets.append(set(args.flatten().tolist()))
    
plt.figure(dpi=300)
plt.title('venn diagram of shared errors')
venn3(sets, model_names[:3])
plt.show()

# ensemble err

In [44]:
preds_test_new = df_preds[[k for k in df_preds if 'proba' in k]].sum(axis=1) / df.shape[1]

In [ ]:
print(f'class imbalance {np.mean(Y_test):0.2f}')
print(f'acc {np.mean(Y_test == preds_labels):0.2f}')
print(f'roc_auc {metrics.roc_auc_score(Y_test, preds_labels):0.2f}')
print(f'balanced acc: {metrics.balanced_accuracy_score(Y_test, preds_labels):0.2f}') #' r2 {metrics.r2_score(Y_train, preds):0.2f}')